# claudette-pydantic

> 

> Adds Pydantic support for [claudette](https://github.com/AnswerDotAI/claudette) through function calling


## Install

```sh
pip install claudette_pydantic
```

## Getting Started

claudette_pydantic patches the `struct` method in the `Client` and `Chat` of claudette

In [ ]:
from claudette.core import *
import claudette_pydantic
from pydantic import BaseModel, Field, ValidationError
from typing import Literal, Union, List, Optional

In [ ]:
model = models[-1]
model

'claude-3-haiku-20240307'

In [ ]:

class Pet(BaseModel):
    "Create a new pet"
    name: str
    age: int
    owner: str = Field(default="NA", description="Owner name. Do not return if not given.")
    type: Literal['dog', 'cat', 'mouse']

In [ ]:
c = Client(model)
c.struct(msgs="Can you make a pet for my dog Mac? He's an old boy (14 years old!)", resp_model=Pet)

Pet(name='Mac', age=14, owner='NA', type='dog')

In [ ]:
c.result.content[0].input

{'name': 'Mac', 'age': 14, 'type': 'dog', 'owner': 'NA'}

In [ ]:
class Cat(BaseModel):
    pet_type: Literal['cat']
    meows: int


class Dog(BaseModel):
    pet_type: Literal['dog']
    barks: float


class Reptile(BaseModel):
    pet_type: Literal['lizard', 'dragon']
    scales: bool


class OwnersPets(BaseModel):
    "List of pets that for an owner"
    pet: List[Union[Cat, Dog, Lizard]] = Field(..., discriminator='pet_type')



In [ ]:
class ChatResponse(BaseModel):
    response_type: Literal['message']
    message: str = Field(description="Support response to the user query.")

class ChatResult(BaseModel):
    response_type: Literal['finish']
    success: bool = Field(description="Respond if the owner has given all the information to check in their pets, *or* cacnelled the process")

class SupportRequest(BaseModel):
    "Respond to a owner request"
    res: Union[ChatResponse, ChatResult] = Field(..., discriminator='response_type')

In [ ]:
chat = Chat(
    models[1],
    sp="You are a vet helpdesk, helping owners check in their pets",
    cont_pr="perform the tool using the information above" # filled as the user prompt when one is not provided (e.g. `Chat.struct(OwnerPets)`)
)
res = chat.struct(SupportRequest, pr="hello I am a new owner and I would like to add some pets for me. I have a dog which has 6 barks, a dragon with 10 scales, and a cat with 2 meows")

res=ChatResponse(response_type='message', message="Hello and welcome! I'd be happy to help you add your pets. However, I need to clarify a few things about the information you've provided. We typically don't measure pets by their barks, scales, or meows. Let's go through your pets one by one to get the information we need:\n\n1. Your dog: We'll need information like its name, breed, age, and any specific care instructions.\n2. Your dragon: I'm afraid we don't actually accommodate dragons as pets in our system. Could you clarify if this is a specific type of reptile or if it's a stuffed animal?\n3. Your cat: Similar to the dog, we'll need its name, breed, age, and any special care instructions.\n\nCould you please provide more details about your dog and cat? And could you clarify what you meant by 'dragon'? Once we have this information, I can help you add your pets to our system.")


OwnersPets(pet=[Dog(pet_type='dog', barks=6.0), Lizard(pet_type='lizard', scales=True), Cat(pet_type='cat', meows=2)])

In [ ]:
print(res)
chat.struct(OwnersPets)